In [1]:
# import MatrixLib
from MatrixLib import *
from importlib import reload
import numpy as np
import numbers
from fractions import Fraction


Общая канва:
* Ввел алгебру (или поле или ВП, не знаю точно) с единицей и нулем, что позволило рекурсивно обрабатывать любую вложенность матриц любых типов
* Ввел <code>\_\_eq__</code>, <code>\_\_neg__</code>,  <code>\_\_truediv__</code>
* <code>\_\_abs__</code> действует аналогично нумпаю
* <code>\_\_sub__</code>, <code>\_\_add__</code>, <code>\_\_rsub__</code> действуют по правилу $A-1 = A- 1E$, где $E$ -- единичный элемент. Такое определение позволило отождествить $E$ и $1$, $0$ и $O$.
* <code>.norm(norm='F')</code> возвращает норму матрицы, F -- фробениуса. Это необходимо, чтобы использовать LUP разложение на волженных матрицах
* Транспонирование <code>.T</code> учитывает вложенность матриц

# 1 Напишите метод lu для класса Matrix, выполняющий LU разложение.

In [2]:
m = FullMatrix(np.array([[2.,2.], [1.,2.]]))
m

| 2.0 2.0 |
| 1.0 2.0 |

In [3]:
a, b = m.lu()
print(a, b ,a*b, sep='\n\n')

| 1   0 |
| 0.5 1 |

| 2.0 2.0 |
| 0   1.0 |

| 2.0 2.0 |
| 1.0 2.0 |


# 2 Реализуйте метод det

Сделал уровень вложенности det

In [4]:
m.det()

2.0

Пример вложенности матриц в матрицу матриц:

In [26]:
d_m = 2
b_m = 2
d_rand = np.random.randint(1, 100, (d_m, b_m, d_m, b_m))
d_array = 1 / d_rand
vf = np.vectorize(lambda x: FullMatrix([[Fraction(1, x)]]))
d_array_frac = vf(d_rand)
d = FullMatrix(np.array([
    [FullMatrix(d_array_frac[i, :, j, :]) for j in range(d_m)] for i in range(d_m)
]))

print(d)
print()
print(d.det(1))
print()
print(d.det(2))
print()
print(d.det(3))

| | | 1/20 | | 1/83 | | | | 1/12 | | 1/39 | | |
| |                   | |                   | |
| | | 1/84 | | 1/62 | | | | 1/45 | | 1/26 | | |
|                                             |
| | | 1/15 | | 1/72 | | | | 1/35 | | 1/11 | | |
| |                   | |                   | |
| | | 1/97 | | 1/83 | | | | 1/30 | | 1/49 | | |

| | -1524825419/387645988800 | | 209710876267/69846054261984 | |
|                                                              |
| | -50347167/67565606080 |    | 47420918005/78261241522464 |  |

| -74906519/512569331588160 |

-74906519/512569331588160


In [27]:
d_m = 2
b_m = 2
d_rand = np.random.randint(1, 100, (d_m, b_m, d_m, b_m))
d_array = 1 / d_rand
vf = np.vectorize(lambda x: Fraction(1, x))
d_array_frac = vf(d_rand)
d = FullMatrix(np.array([
    [FullMatrix(d_array_frac[i, :, j, :]) for j in range(d_m)] for i in range(d_m)
]))

print(d)
print()
print(d.det(1))
print()
print(d.det(2))
print(d.det(3))

| | 1/99 1/21 | | 1/49 1/71 | |
| | 1/97 1/11 | | 1/12 1/74 | |
|                             |
| | 1/49 1/41 | | 1/9  1/24 | |
| | 1/5  1/81 | | 1/24 1/11 | |

| 17811288091/356356982520     -433934900891/44578561575240       |
| 1081244729099/11522209101480 -296313429221969/15855108400260360 |

-8613386639719/417250257773940360
-8613386639719/417250257773940360


# Убедитесь, что ваша реализация LU разложения работает с блочными матрицами.

In [6]:
b = FullMatrix.zero(2,2,m)
b

| | 2.0 2.0 | | 2.0 2.0 | |
| | 1.0 2.0 | | 1.0 2.0 | |
|                         |
| | 2.0 2.0 | | 2.0 2.0 | |
| | 1.0 2.0 | | 1.0 2.0 | |

In [7]:
print(m)
print()
print(m.inverse().inverse())

| 2.0 2.0 |
| 1.0 2.0 |

| 2.0 2.0 |
| 1.0 2.0 |


In [8]:
l, u = d.lu()
print(d, l, u, d == l * u, sep ='\n\n')

| | 1/9  1/15 | | 1/10 1/14 | |
| | 1/40 1/17 | | 1/68 1/8  | |
|                             |
| | 1/17 1/84 | | 1/68 1/71 | |
| | 1/75 1/61 | | 1/74 1/75 | |

| | 1 0 |                    | 0 0 | |
| | 0 1 |                    | 0 0 | |
|                                    |
| | 46065/70924 -1670/3129 | | 1 0 | |
| | 699/9089    8704/45445 | | 0 1 | |

| | 1/9  1/15 | | 1/10 1/14 |                       |
| | 1/40 1/17 | | 1/68 1/8  |                       |
|                                                   |
| | 0 0 |       | -18041/425544 7276793/211495368 | |
| | 0 0 |       | 1011/336293   -2519/156450      | |

True


# Реализуйте LUP разложение с перестановкой строк
Предъявите матрицу, на которой LUP разложение работает, а LU - нет.

In [9]:
test_a = FullMatrix([[0,1], [2,3]])
test_a.lu()

C:\Users\vigos\JupyterProjects\MatrixLabs\MatrixLabs2022\MatrixLib\Matrixs.py:10: RuntimeWarning: divide by zero encountered in long_scalars
  return 1 / element


(| 1   0 |
 | inf 1 |,
 | 0 1    |
 | 0 -inf |)

In [10]:
test_a.lup()

(| 1   0 |
 | 0.0 1 |,
 | 2 3   |
 | 0 1.0 |,
 | 0 1 |
 | 1 0 |)

# Реализуйте метод прогонки и реализуйте метод Matrix.solve для решения линейных систем уравнений.

Решим d*x = B

In [11]:
print(d)

| | 1/9  1/15 | | 1/10 1/14 | |
| | 1/40 1/17 | | 1/68 1/8  | |
|                             |
| | 1/17 1/84 | | 1/68 1/71 | |
| | 1/75 1/61 | | 1/74 1/75 | |


In [12]:
B = FullMatrix([[1], [0]])
B

| 1 |
| 0 |

In [13]:
x = d.solve(B)
x

| | -126848071350/41469422587 -27506346120/41469422587  | |
| | -973833416640/41469422587 -238799613360/41469422587 | |
|                                                         |
| | 938242604880/41469422587 -143459451760/41469422587 |  |
| | 373262680350/41469422587 466510521120/41469422587  |  |

In [14]:
d * x

| | 1 0 | |
| | 0 1 | |
|         |
| | 0 0 | |
| | 0 0 | |

Совпало с b

# Реализуйте класс SymmetricMatrix, хранящий симметричные матрицы. Убедитесь, что метод Matrix.lu корректно работает с этим классом. Модифицируйте этот метод для класса SymmetricMatrix так, чтобы он использовал симметричность матрицы и работал в два раза быстрее.

Получилось. Чтобы создать матрицу, подается весь нумпай массив. По аргументу side='u' или 'd' используется либо верхняя часть массива для заполнения, либо нижняя. При этом блочные матрицы транспонируются, а матрицы, стоящие на диагонали сами становятся экземплярами SymmetricMatrix с тем же параметром side

In [21]:
d_m =2
b_m =2
d_rand = np.random.randint(1, 100, (d_m, b_m, d_m, b_m))
d_array = 1 / d_rand
vf = np.vectorize(lambda x: Fraction(x,1))
d_array_frac = vf(d_rand)
d = SymmetricMatrix(np.array([
    [FullMatrix(d_array_frac[i, :, j, :]) for j in range(d_m)] for i in range(d_m)
]))

L, D = SymmetricMatrix(d.data).ldl()

In [22]:
print(d, L, D, L * D * L.T == d, sep='\n\n')

| | 45 67 | | 45 57 | |
| | 67 4  | | 19 18 | |
|                     |
| | 45 19 | | 9  96 | |
| | 57 18 | | 96 5  | |

| | 1 0 |                 | 0 0 | |
| | 0 1 |                 | 0 0 | |
|                                 |
| | 1093/4309 2160/4309 | | 1 0 | |
| | 978/4309  3009/4309 | | 0 1 | |

| | 45 67 | | 0 0 |                     |
| | 67 4  | | 0 0 |                     |
|                                       |
| | 0 0 |   | -51444/4309 312483/4309 | |
| | 0 0 |   | 312483/4309 -88363/4309 | |

True


# Реализуйте класс BandMatrix для хранения ленточных матриц. Убедитесь в работоспособности методов lu и solve.

Не получилось чуть-чуть: где-то случается перестановка, которую я не могу отловить:

In [40]:
p = {
    0: [Fraction(1), Fraction(6), Fraction(1), Fraction(1)],
    1: [1, Fraction(1), 3],
    -1: [Fraction(6), 2, 2],
}
BM = BandMatrix(p)
print(BM)
print('Не выходит E:')
print(BM * BM.inverse())

| 1 1 0 0 |
| 6 6 1 0 |
| 0 2 1 3 |
| 0 0 2 1 |
Не выходит E:
| 0 0 1 0 |
| 0 0 0 1 |
| 1 0 0 0 |
| 0 1 0 0 |


In [52]:
def block_FullMatrix(m: int, n: int, start_from: int = 1):
    a = np.arange(start_from, start_from + m*n).astype(float).reshape((m, n))
    return FullMatrix(a)

p = {
    0: [block_FullMatrix(2, 2, np.random.randint(0,10)) for i in range(4)],
    1: [block_FullMatrix(2, 2, np.random.randint(0,10)) for i in range(3)],
    -2: [block_FullMatrix(2, 2, np.random.randint(0,10)) for i in range(2)],
}

In [53]:
BandMatrix(p)

| | 5.0 6.0 |   | 6.0 7.0 | 0           0             |
| | 7.0 8.0 |   | 8.0 9.0 |                           |
|                                                     |
| 0             | 4.0 5.0 | | 5.0 6.0 | 0             |
|               | 6.0 7.0 | | 7.0 8.0 |               |
| | 8.0  9.0  | 0           | 3.0 4.0 | | 9.0  10.0 | |
| | 10.0 11.0 |             | 5.0 6.0 | | 11.0 12.0 | |
|                                                     |
| 0             | 5.0 6.0 | 0           | 7.0 8.0  |  |
|               | 7.0 8.0 |             | 9.0 10.0 |  |